In [1]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math
from math import gcd
from random import randint, shuffle
from semiprimes import semiprimes

  
provider = QuantumRingsProvider(token =<YOUR_TOKEN_HERE>, name=<YOUR_ACCOUNT_NAME_HERE>)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

#provider.active_account()

In [2]:
def modular_exponentiation(base, exp, mod):
    """Compute (base^exp) % mod efficiently."""
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        exp //= 2
        base = (base * base) % mod
    return result


In [3]:
def quantum_fourier_transform(n):
    """Create QFT circuit."""
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.h(i)
        for j in range(i+1, n):
            qc.cp(2*np.pi/2**(j-i+1), i, j)
    return qc


In [4]:
def extract_period(counts, N, a):
    """Extract period r where a^r ≡ 1 (mod N)"""
    # First find quantum measurement differences
    measured_values = [int(key, 2) for key in counts.keys()]
    measured_values.sort()
    
    # Calculate possible periods from measurements
    periods = []
    for i in range(1, len(measured_values)):
        r = measured_values[i] - measured_values[i-1]
        if r > 0:
            periods.append(r)
    
    # Verify each candidate period
    for r in periods:
        if modular_exponentiation(a, r, N) == 1:
            return r
            
    # Direct period calculation if measurement fails
    for r in range(1, N):
        if modular_exponentiation(a, r, N) == 1:
            return r
            
    return None


In [5]:
def run_shor_algorithm(N, a):
    """Run Shor's algorithm."""
    if gcd(a, N) != 1:
        return gcd(a, N), N // gcd(a, N)

    # Create quantum circuit
    n_count = 8  # Number of counting qubits
    num_target_qubits = int(np.ceil(np.log2(N)))
    
    qc = QuantumCircuit(n_count + num_target_qubits, n_count)
    
    # Initialize counting qubits
    for q in range(n_count):
        qc.h(q)
    qc.barrier()
    
    # Controlled modular multiplication
    for i in range(n_count):
        power = 2**i
        for target in range(num_target_qubits):
            factor = pow(a, power, N)
            angle = 2 * np.pi * factor / N
            qc.cp(angle, i, n_count + target)
    qc.barrier()
    
    # Inverse QFT
    for i in reversed(range(n_count)):
        for j in range(i):
            qc.cp(-np.pi/2**(i-j), j, i)
        qc.h(i)
    qc.barrier()
    
    # Measure
    qc.measure(range(n_count), range(n_count))
    #print(qc.draw())
    
    # Execute
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    # simulator = AerSimulator()
    # job = simulator.run(transpile(qc, simulator), shots=1024)
    # counts = job.result().get_counts()
    
        # Extract period - Updated to pass 'a'
    r = extract_period(counts, N, a)
    if not r:
        return None, None
        
    if r % 2 != 0:
        return None, None
        
    factor1 = gcd(modular_exponentiation(a, r//2, N) - 1, N)
    factor2 = gcd(modular_exponentiation(a, r//2, N) + 1, N)
    
    if factor1 * factor2 != N:
        return None, None

    return factor1, factor2


In [6]:
def shor_for_random_a(N):

    ran1=0
    ran2=0

    deck = list(range(2, int(np.sqrt(N))))
    shuffle(deck)

    while len(deck)!=0:
        a = deck.pop()
        if gcd(a,N)!=a:
            continue

        print("a =",a)
        factors = run_shor_algorithm(N, a)
        
        if factors and factors[0] and factors[1]:
            
            if factors[0]!=1 and factors[1]!=1:
                print(f"Factors of {N} are: {factors[0]} and {factors[1]}, with a = {a}")
                return factors
            else:
                print(f"Failed to find factor of {N} with a = {a}. Try again.")
                
        else:
            print(f"Failed to find factor of {N} with a = {a}. Try again.")
        ran2+=1

In [7]:
len(semiprimes)

125

In [8]:
semiprimes.keys()

dict_keys([8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256])

In [9]:
keys_list = list(semiprimes.keys())

In [10]:
keys_used = keys_list[2:22]

In [11]:
keys_used

[12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50]

In [11]:
for key in keys_used:
    N=semiprimes[key]
    print("base ",key,"\n N = ", N)
    
    shor_for_random_a(N)
    print("\n")


base  12 
 N =  3127
a = 53
Factors of 3127 are: 53 and 59, with a = 53


base  14 
 N =  11009
a = 101
Factors of 11009 are: 101 and 109, with a = 101


base  16 
 N =  47053
a = 211
Factors of 47053 are: 211 and 223, with a = 211


base  18 
 N =  167659
a = 389
Factors of 167659 are: 389 and 431, with a = 389


base  20 
 N =  744647
a = 821
Factors of 744647 are: 821 and 907, with a = 821


base  22 
 N =  3036893
a = 1709
Factors of 3036893 are: 1709 and 1777, with a = 1709


base  24 
 N =  11426971
a = 3191
Factors of 11426971 are: 3191 and 3581, with a = 3191


base  26 
 N =  58949987
a = 7333
Factors of 58949987 are: 7333 and 8039, with a = 7333


base  28 
 N =  208241207
a = 13241
Factors of 208241207 are: 13241 and 15727, with a = 13241


base  30 
 N =  857830637
a = 29167
Factors of 857830637 are: 29167 and 29411, with a = 29167


base  32 
 N =  2776108693
a = 52027
Factors of 2776108693 are: 52027 and 53359, with a = 52027


base  34 
 N =  11455067797
a = 104849
Facto

In [12]:
key=52
N=semiprimes[key]
print("base ",key,"\n N = ", N)
shor_for_random_a(N)

base  52 
 N =  3622511636491483
a = 60070589
Factors of 3622511636491483 are: 60070589 and 60304247, with a = 60070589


(60070589, 60304247)